In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/My Drive/cifar10_project')
os.getcwd()

'/content/drive/My Drive/cifar10_project'

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

W0817 11:11:10.755305 140325568321408 deprecation.py:323] From <ipython-input-3-46c62fe1e279>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0817 11:11:10.757248 140325568321408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0817 11:11:10.762649 140325568321408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting ./mnist/data/train-images-idx3-ubyte.gz


W0817 11:11:11.024872 140325568321408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0817 11:11:11.030142 140325568321408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0817 11:11:11.082495 140325568321408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [0]:
import tensorflow as tf
  
class GAN_dense():
    def __init__(self, input_shape = [784], n_noise=128):
    
        self.input_shape = input_shape
        self.n_noise = n_noise
        
    def get_noise(self, batch_size):
        return np.random.uniform(-1.,1., size = [batch_size, self.n_noise])

    def generator(self, noise, labels):
        with tf.variable_scope('generator',reuse = tf.AUTO_REUSE):
            
            inputs = tf.concat([noise,labels], 1)
            
            x = tf.layers.dense(inputs, 256, activation=tf.nn.relu)
            x = tf.layers.dense(x, 784, activation=tf.nn.sigmoid)
            x = tf.reshape(x,[-1]+self.input_shape)
            
            return x
        
    def discriminator(self, inputs, labels):
        with tf.variable_scope('discriminator',reuse = tf.AUTO_REUSE):
            
            x = tf.layers.flatten(inputs)
            x = tf.concat([x, labels], 1)
            x = tf.layers.dense(x, 256, activation = tf.nn.relu)
            x = tf.layers.dense(x, 1, activation = None)
            
            return x     

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.reset_default_graph()

input_shape = [784]
n_class = 10
n_noise=128

model = GAN_dense(input_shape = input_shape, n_noise = n_noise)


X = tf.placeholder(tf.float32, [None]+input_shape)
Y = tf.placeholder(tf.float32, [None,n_class])
Z = tf.placeholder(tf.float32, [None,n_noise])

G = model.generator(Z,Y)
D_real = model.discriminator(X, Y) 
D_gene = model.discriminator(G,Y)
                
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels=tf.zeros_like(D_gene)))
loss_D = loss_D_real + loss_D_gene
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels = tf.ones_like(D_gene)))
        
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                  scope = 'discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                  scope = 'generator')
train_D = tf.train.AdamOptimizer().minimize(loss_D,
                                             var_list = vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G,
                                             var_list = vars_G)

In [8]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

total_epoch = 50
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        noise = model.get_noise(batch_size)

        _, cost1 = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, cost2 = sess.run([train_G, loss_G],
                                 feed_dict={Y: batch_ys, Z: noise})
        loss_val_D += cost1/total_batch
        loss_val_G += cost2/total_batch
        
    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))

    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = model.get_noise(sample_size)
        samples = sess.run(G,
                           feed_dict={Y: mnist.test.labels[:sample_size],
                                      Z: noise})

        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))

        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()

            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples/dense_mnist_{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('done')

Epoch: 0000 D loss: 0.05668 G loss: 5.896
Epoch: 0001 D loss: 0.06678 G loss: 13.54
Epoch: 0002 D loss: 0.08265 G loss: 21.54
Epoch: 0003 D loss: 0.0996 G loss: 28.5
Epoch: 0004 D loss: 0.1233 G loss: 36.02
Epoch: 0005 D loss: 0.1488 G loss: 43.54
Epoch: 0006 D loss: 0.1841 G loss: 51.72
Epoch: 0007 D loss: 0.2446 G loss: 58.91
Epoch: 0008 D loss: 0.3278 G loss: 65.91
Epoch: 0009 D loss: 0.4335 G loss: 72.63
Epoch: 0010 D loss: 0.5858 G loss: 78.77
Epoch: 0011 D loss: 0.8076 G loss: 84.39
Epoch: 0012 D loss: 1.083 G loss: 89.51
Epoch: 0013 D loss: 1.403 G loss: 94.09
Epoch: 0014 D loss: 1.791 G loss: 98.5
Epoch: 0015 D loss: 2.206 G loss: 102.8
Epoch: 0016 D loss: 2.658 G loss: 106.8
Epoch: 0017 D loss: 3.164 G loss: 110.5
Epoch: 0018 D loss: 3.706 G loss: 113.9
Epoch: 0019 D loss: 4.245 G loss: 117.2
Epoch: 0020 D loss: 4.821 G loss: 120.4
Epoch: 0021 D loss: 5.42 G loss: 123.7
Epoch: 0022 D loss: 6.038 G loss: 126.6
Epoch: 0023 D loss: 6.678 G loss: 129.5
Epoch: 0024 D loss: 7.34 G l